In [7]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

In [10]:
def read_data(file_name):
    df = pd.read_csv(file_name, index_col=0, parse_dates=True)
    return df

file_name = 'data/BTCUSDT_15m_20210801_20211231.csv'
ohlcv_15m = read_data(file_name)
ohlcv_15m

,open,high,low,close,volume,symbol,timeframe
timestamp,,,,,,,
2021-08-01 00:15:00,41471.68,41646.94,41422.70,41638.83,0.097362,BTC/USDT,15m
2021-08-01 00:30:00,41638.83,41697.96,41549.33,41686.15,0.150340,BTC/USDT,15m
2021-08-01 00:45:00,41686.15,41773.13,41664.35,41727.69,0.446030,BTC/USDT,15m
2021-08-01 01:00:00,41727.69,41902.45,41669.21,41879.14,0.993220,BTC/USDT,15m
2021-08-01 01:15:00,41879.14,41911.40,41660.55,41715.40,1.040230,BTC/USDT,15m
...,...,...,...,...,...,...,...
2021-12-31 19:00:00,46662.65,46665.92,46386.32,46427.41,21.564411,BTC/USDT,15m
2021-12-31 19:15:00,46427.41,46431.66,46075.28,46087.99,20.032880,BTC/USDT,15m
2021-12-31 19:30:00,46087.99,46322.23,45987.40,46084.71,40.999063,BTC/USDT,15m


In [11]:
def preprocess_data(df):
    # 特徴量の追加
    df["return"] = df["close"].pct_change()
    df["return_std"] = df["return"].rolling(window=10).std()
    
    # ラベルの作成
    df["future_return"] = df["close"].pct_change().shift(-1)
    
    # 欠損値の削除
    df.dropna(inplace=True)
    
    # 特徴量とラベルの分離
    X = df.drop("future_return", axis=1)
    y = df["future_return"]
    
    return X, y

X, y = preprocess_data(ohlcv_15m)

In [12]:
def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=False)
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_test = lgb.Dataset(X_test, y_test, reference=lgb_train)

    params = {
        'objective': 'regression',
        'metric': 'mse',
        'boosting_type': 'gbdt',
        'learning_rate': 0.01,
        'num_leaves': 31,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': 0
    }

    model = lgb.train(params,
                      lgb_train,
                      num_boost_round=10000,
                      valid_sets=lgb_test,
                      early_stopping_rounds=100,
                      verbose_eval=False)

    return model, X_train, X_test, y_train, y_test

model, X_train, X_test, y_train, y_test = train_model(X, y)

/Users/tek/dev/python/learning-model/myenv/lib/python3.11/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/tek/dev/python/learning-model/myenv/lib/python3.11/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: symbol, timeframe

In [ ]:
rsTrSja7mUyP4s2oeM
xJneAAMp8OmbNVJlZPnaKvy7rP5ByVIlhzmB